- Hashed Features Design Pattern

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery df --project project_name
CREATE TEMPORARY FUNCTION hashed(airport STRING, numbuckets INT64) AS (
   ABS(MOD(FARM_FINGERPRINT(airport), numbuckets))
);

WITH airports AS (
SELECT 
   DISTINCT(departure_airport)
FROM `bigquery-samples.airline_ontime_data.flights`
)

SELECT 
   departure_airport,
   hashed(departure_airport, 3) AS hash3,
   hashed(departure_airport, 10) AS hash10,
   hashed(departure_airport, 1000) AS hash1000,
FROM airports

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
df.describe()

,hash3,hash10,hash1000
count,347.000000,347.000000,347.000000
mean,1.023055,4.440922,505.622478
std,0.832531,2.788052,277.258271
min,0.000000,0.000000,8.000000
25%,0.000000,2.000000,296.500000
50%,1.000000,4.000000,508.000000
75%,2.000000,7.000000,726.500000
max,2.000000,9.000000,999.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347 entries, 0 to 346
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   departure_airport  347 non-null    object
 1   hash3              347 non-null    Int64 
 2   hash10             347 non-null    Int64 
 3   hash1000           347 non-null    Int64 
dtypes: Int64(3), object(1)
memory usage: 12.0+ KB


In [10]:
df.head(n=10)

,departure_airport,hash3,hash10,hash1000
0,ATL,0,8,838
1,IAH,2,9,209
2,SAT,0,4,764
3,ELP,2,3,193
4,TUS,1,6,666
5,LAS,1,6,706
6,SEA,2,7,597
7,TUL,1,8,278
8,HOU,0,9,659
9,SAN,0,7,447


In [11]:
print(f"length: {len(df)}")

length: 347


Some airports had very few flights

In [13]:
%%bigquery --project project_name
SELECT 
   departure_airport, COUNT(1) AS num_flights
FROM `bigquery-samples.airline_ontime_data.flights`
GROUP BY departure_airport
ORDER BY num_flights ASC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,departure_airport,num_flights
0,BFF,1
1,MKC,1
2,PVU,1
3,GLH,2
4,FMN,3
5,PUB,4
6,OGD,5
7,CKB,8
8,PIR,14
9,SHD,18


- Likelihood of collision

In [14]:
import pandas as pd

def calc_collision_prob(num_total, num_hash):
    no_collision_prob = 1.0
    for i in range(num_total):
        # i of the previous buckets is occupied now
        collision_likelihood = float(i) / num_hash
        no_collision_prob *= (1 - collision_likelihood)
    return 1 - no_collision_prob


data = []
for num_hash in [3, 10, 100, 1000, 10000, 100000]:
    data.append([num_hash, 
                 len(df)/num_hash, 
                 calc_collision_prob(len(df), num_hash)
                ])
prob = pd.DataFrame(data, columns=['num_hash_buckets', 'entries_per_bucket', 'collision_prob'])
prob

,num_hash_buckets,entries_per_bucket,collision_prob
0,3,115.666667,1.000000
1,10,34.700000,1.000000
2,100,3.470000,1.000000
3,1000,0.347000,1.000000
4,10000,0.034700,0.997697
5,100000,0.003470,0.451739


In [15]:
calc_collision_prob(5, 1000) # num_hash >> num_total

0.009965049976000118

- Airports that share hash buckets

In [16]:
%%bigquery --project project_name
CREATE TEMPORARY FUNCTION hashed(airport STRING, numbuckets INT64) AS (
   ABS(MOD(FARM_FINGERPRINT(airport), numbuckets))
);

WITH airports AS (
SELECT 
   departure_airport, COUNT(1) AS num_flights
FROM `bigquery-samples.airline_ontime_data.flights`
GROUP BY departure_airport 
)

SELECT 
   departure_airport, num_flights
FROM airports
WHERE hashed(departure_airport, 100) = hashed('ORD', 100)

Query is running:   0%|          |

Downloading:   0%|          |

,departure_airport,num_flights
0,ORD,3610491
1,MCI,597761
2,BTV,66555
